In [1]:
from lib.functions.get_temperature_path import get_temperatures_path

last_fetch_path = get_temperatures_path()
last_fetch_path

PosixPath('/home/naheridi/DataspellProjects/temperature-analysis/notebooks/../data/temperatures/2021-04-12T13:38:31')

In [2]:
from pandas import read_csv

temperatures = read_csv(last_fetch_path / '2_normalized' / 'filtered_dataset.csv', parse_dates=True, index_col=0)
temperatures

,wall_316,wall_210,bat_412_a,bat_316,bat_420,wall_412_a,bat_210,wall_420
2020-03-02 09:00:00,22.62,NaN,43.06,47.06,50.25,23.44,NaN,22.44
2020-03-02 09:10:00,22.62,24.25,43.19,46.81,49.88,23.50,43.50,22.44
2020-03-02 09:20:00,22.56,23.06,43.00,46.63,49.31,23.44,43.00,22.25
2020-03-02 09:30:00,22.62,21.94,42.69,46.56,49.31,23.50,42.06,22.69
2020-03-02 09:40:00,22.62,22.25,42.56,NaN,49.38,23.44,42.13,22.94
...,...,...,...,...,...,...,...,...
2021-12-04 12:50:00,19.94,24.81,23.00,41.37,46.81,21.44,40.00,22.56
2021-12-04 13:00:00,19.94,24.87,23.00,37.31,45.63,21.50,40.69,22.56
2021-12-04 13:10:00,20.06,24.87,23.06,39.94,49.31,21.56,39.88,22.62
2021-12-04 13:20:00,20.12,24.87,23.25,39.00,45.81,21.75,40.44,22.62


In [3]:
wall_temperatures = temperatures[[column for column in temperatures.columns if column.startswith('wall')]]
wall_temperatures = wall_temperatures.rename(columns={column: column.replace('wall', '').replace('_', '') for column in wall_temperatures.columns})

Вычислим месячную функцию автокорреляции

In [4]:
from statsmodels.tsa.stattools import acf
from pandas import DataFrame

monthly = wall_temperatures.resample('M').mean()
monthly_acfs = [acf(monthly[column], missing='drop', alpha=0.05) for column in monthly.columns]
monthly_acf_df = DataFrame(
    [monthly_acf[0] for monthly_acf in monthly_acfs],
    index=monthly.columns
).T
monthly_acf_df

,316,210,412a,420
0,1.000000,1.000000,1.000000,1.000000
1,0.532181,0.307042,0.339462,0.472210
2,0.135367,-0.056034,-0.076096,0.105404
3,-0.047530,0.120626,0.019852,0.049297
4,-0.383111,-0.207625,-0.301671,-0.313158
5,-0.362157,-0.324196,-0.275492,-0.365963
6,-0.137749,-0.109475,0.174708,-0.195175
7,-0.167789,-0.171252,-0.021495,-0.225815
8,-0.155761,-0.218141,-0.167666,-0.168416
9,-0.006822,-0.018104,0.009821,0.029835


In [5]:
analysis_path = last_fetch_path / '3_analysis'
analysis_path.mkdir(parents=True, exist_ok=True)

In [11]:
from plotly.subplots import make_subplots
from numpy import arange

fig = make_subplots(rows=len(monthly_acfs), subplot_titles=monthly_acf_df.columns)

for i, monthly_acf in enumerate(monthly_acfs):
    x = arange(len(monthly_acf[0]))
    fig.add_bar(
        x=x,
        y=monthly_acf[0],
        showlegend=False,
        name=monthly_acf_df.columns[i],
        row=i + 1,
        col=1
    )
    fig.add_scatter(
        x=x,
        y=monthly_acf[1][:, 1] - monthly_acf[0],
        mode='lines',
        line_color='rgba(0, 0, 0, 0)',
        showlegend=False,
        name=f'▲{monthly_acf_df.columns[i]}',
        row=i + 1,
        col=1
    )
    fig.add_scatter(
        x=x,
        y=monthly_acf[1][:, 0] - monthly_acf[0],
        mode='none',
        showlegend=False,
        fill='tonextx',
        name=f'▼{monthly_acf_df.columns[i]}',
        row=i + 1,
        col=1
    )
fig.update_layout(
    title='Автокорреляционная функция',
    template='seaborn',
    height=720
)
fig.write_image(analysis_path / 'autocorrelation.png', width=1080)
fig.show()

In [55]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=4)
resampled = wall_temperatures.resample('D').mean()

for i, x_col in enumerate(wall_temperatures.columns):
    for j, y_col in enumerate(wall_temperatures.columns):
        inner_corr = resampled[x_col].rolling(90, min_periods=10).corr(resampled[y_col])
        fig.add_scatter(
            x=inner_corr.index,
            y=[round(value, 2) for value in inner_corr],
            mode='lines',
            name=f'{x_col}, {y_col}',
            row=i + 1,
            col=j + 1
        )

fig.update_layout(
    title='Корреляция',
    template='seaborn',
    height=720
)
fig.write_image(analysis_path / 'correlation.png', width=1080)
fig.show()